# ENet -  Real Time Semantic Segmentation for Road segmentation

**Journal paper:**<br/>
Link to the paper: https://arxiv.org/pdf/1606.02147.pdf <br/>
**Code references**<br/>
python implementation :https://github.com/iArunava/ENet-Real-Time-Semantic-Segmentation<br/>
lua implemenattion by paper author : https://github.com/e-lab/ENet-training

Daya Alex 5/4/24

## Wandb login

In [ ]:
import wandb
wandb.login()

## Install the dependencies and Import them

In [ ]:
#!pip install pytorch-lightning

In [ ]:
pip install segmentation_models_pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
from torchvision import transforms
import math
import random
from torchmetrics import JaccardIndex
import albumentations as A
from torch.utils.data import DataLoader, Dataset
from segmentation_models_pytorch.losses import DiceLoss


#from torch.optim.lr_scheduler import StepLR
#import torch.optim.lr_scheduler as lr_scheduler
from PIL import Image



## Create the ENet model

We decided to to split the model to three sub classes:

1) Initial block  

2) RDDNeck - class for regular, downsampling and dilated bottlenecks

3) ASNeck -  class for asymetric bottlenecks

4) UBNeck - class for upsampling bottlenecks

In [ ]:
class InitialBlock(nn.Module):
  
  # Initial block of the model:
  #         Input
  #        /     \
  #       /       \
  # maxpool2d    conv2d-3x3
  #       \       /  
  #        \     /
  #      concatenate
  #          |
  #         Batchnorm
 #        PReLU
   
    def __init__ (self,in_channels = 3,out_channels = 13):
        super().__init__()


        self.maxpool = nn.MaxPool2d(kernel_size=2, 
                                      stride = 2, 
                                      padding = 0)

        self.conv = nn.Conv2d(in_channels, 
                                out_channels,
                                kernel_size = 3,
                                stride = 2, 
                                padding = 1)

        self.prelu = nn.PReLU(16)

        self.batchnorm = nn.BatchNorm2d(16)
  
    def forward(self, x):
        
        main = self.conv(x)
        side = self.maxpool(x)
        #print('main size ', main.size)
        #print('side size ', side.size)
        # concatenating on the channels axis
        x = torch.cat((main, side), dim=1)
        x = self.batchnorm(x)
        x = self.prelu(x)
        #print('init block size ',x.shape)
        
        return x

In [ ]:
class RDDNeck(nn.Module):
    def __init__(self, dilation, in_channels, out_channels, down_flag, relu=False, projection_ratio=4, p=0.1):
    
  # Regular|Dilated|Downsampling bottlenecks:
  #
  #     Bottleneck Input
  #        /        \
  #     Identity     \  
  #       /          \
  # maxpooling2d   conv2d-1x1(when downsamp flag is ON, otherwise 2x2)
  # (when downsamp)    | BN +PReLU
  # (-flag is ON)    conv2d-3x3
  #      |             | BN +PReLU
  #      |         conv2d-1x1
  #      |             |
  #  Padding2d     Regularizer(BN + dropout)
  #(when i/p ch !=o/p ch) /   
  #        \            /
  #      Summing + PReLU
  #
  # Params: 
  #  dilation (bool) - if True: creating dilation bottleneck
  #  down_flag (bool) - if True: creating downsampling bottleneck
  #  projection_ratio - ratio between input and output channels
  #  relu - if True: relu used as the activation function else: Prelu us used
  #  p - dropout ratio
        
        super().__init__()
        
        # Define class variables
        self.in_channels = in_channels
        self.reduced_depth = int(out_channels // projection_ratio)
        self.out_channels = out_channels
        self.dilation = dilation
        self.down_flag = down_flag
        
        # calculating the number of reduced channels
        if self.down_flag:
            self.stride = 2
            self.conv1_kernel = 2
        else:
            self.stride = 1
            self.conv1_kernel = 1
        
        if relu:
            activation = nn.ReLU()
        else:
            activation = nn.PReLU()
        
        self.maxpool = nn.MaxPool2d(kernel_size = 2,
                                      stride = 2,
                                      padding = 0, return_indices=True)
        
        self.dropout = nn.Dropout2d(p=p)

        self.prelu1 = activation
        
        self.conv1 = nn.Conv2d(in_channels = self.in_channels,
                               out_channels = self.reduced_depth,
                               kernel_size = self.conv1_kernel,
                               stride = self.stride,
                               padding = 0,
                               bias = False,
                               dilation = 1)
        
        self.conv2 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.reduced_depth,
                                  kernel_size = 3,
                                  stride = 1,
                                  padding = self.dilation,
                                  bias = True,
                                  dilation = self.dilation)
                                  
        self.prelu2 = activation
        
        self.conv3 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.out_channels,
                                  kernel_size = 1,
                                  stride = 1,
                                  padding = 0,
                                  bias = False,
                                  dilation = 1)
        
        self.prelu3 = activation
        
        self.batchnorm = nn.BatchNorm2d(self.reduced_depth)
        self.batchnorm2 = nn.BatchNorm2d(self.out_channels)
        self.identity = nn.Identity()
        
    def forward(self, x):
        
        bs = x.size()[0]
        x_copy = self.identity(x)
        
        # Main
        x = self.conv1(x)
        #print(" Conv1 called ")
        x = self.batchnorm(x)
        x = self.prelu1(x)
        #print(self.conv1_kernel, self.stride, x.shape)
        
        x = self.conv2(x)
        #print(" Conv2 called ")
        x = self.batchnorm(x)
        x = self.prelu2(x)
        #print(self.conv2.kernel_size, self.stride,self.dilation, x.shape)
        
        x = self.conv3(x)
        #print(" Conv3 called ")
        x = self.batchnorm2(x)     
        x = self.dropout(x)
        #print(self.conv3.kernel_size, self.stride, x.shape)
        
        #other
        if self.down_flag:
            #print('downsampling flag is true')
            x_copy, indices = self.maxpool(x_copy)
            
          
        if self.in_channels != self.out_channels:
            #print('input and output channels diffrence, so padding of side channel being carried out')
            out_shape = self.out_channels - self.in_channels
            #print('extra channels required ', out_shape)
            
            #padding and concatenating in order to match the channels axis of the side and main branches
            extras = torch.zeros((bs, out_shape, x.shape[2], x.shape[3]))
            if torch.cuda.is_available():
                extras = extras.cuda()
            #elif torch.backends.mps.is_available():
                #extras = extras.to('mps')
            #print('x copy shape ',x_copy.shape)
            #print('extras shape ', extras.shape)
            x_copy = torch.cat((x_copy, extras), dim = 1)
            #print('final side route shape ,', x_copy.shape)

        # Summing main and side branches
        x = x + x_copy
        x = self.prelu3(x)
        #print('final layer ', x.shape)
        
        if self.down_flag:
            return x, indices
        else:
            return x

In [ ]:
class ASNeck(nn.Module):
    def __init__(self, in_channels, out_channels, projection_ratio=4):
      
  # Asymetric bottleneck:
  #
  #     Bottleneck Input
  #        /        \
  #       /          \
  #      |         conv2d-1x1
  # Identity           | PReLU
  #      |         conv2d-1x5
  #      |             |
  #      |         conv2d-5x1
  #      |             | PReLU
  #      |         conv2d-1x1
  #      |             |
  #       \     Regularizer
  #       \           /  
  #        \         /
  #      Summing + PReLU
  #
  # Params:    
  #  projection_ratio - ratio between input and output channels
        
        super().__init__()
        
        # Define class variables
        self.in_channels = in_channels
        self.reduced_depth = int(out_channels // projection_ratio)
        self.out_channels = out_channels
        
        self.dropout = nn.Dropout2d(p=0.1)
        
        self.conv1 = nn.Conv2d(in_channels = self.in_channels,
                               out_channels = self.reduced_depth,
                               kernel_size = 1,
                               stride = 1,
                               padding = 0,
                               bias = False)
        
        self.prelu1 = nn.PReLU()
        
        self.conv21 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.reduced_depth,
                                  kernel_size = (5, 1),
                                  stride = 1,
                                  padding = (2, 0),
                                  bias = False)
        
        self.conv22 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.reduced_depth,
                                  kernel_size = (1, 5),
                                  stride = 1,
                                  padding = (0, 2),
                                  bias = True)############TRUE bias in original code###############
        
        self.prelu2 = nn.PReLU()
        
        self.conv3 = nn.Conv2d(in_channels = self.reduced_depth,
                                  out_channels = self.out_channels,
                                  kernel_size = 1,
                                  stride = 1,
                                  padding = 0,
                                  bias = False)
        
        self.prelu3 = nn.PReLU()
        
        self.batchnorm = nn.BatchNorm2d(self.reduced_depth)
        self.batchnorm2 = nn.BatchNorm2d(self.out_channels)
        self.identity = nn.Identity()
        
    def forward(self, x):
   
        #main branch
        x_copy = self.identity(x)
        #print('side branch')
        
        # Side Branch
        x = self.conv1(x)
        #print('conv1 called')
        x = self.batchnorm(x)
        x = self.prelu1(x)
        #print(self.conv1.kernel_size,self.conv1.stride, x.shape)
        
        x = self.conv21(x)
        #print('conv21 called')
        #print(self.conv21.kernel_size, self.conv21.stride, x.shape)
        x = self.conv22(x)
        #print('conv22 called')
        #print(self.conv22.kernel_size, self.conv22.stride, x.shape)
        x = self.batchnorm(x)
        x = self.prelu2(x)
        
        x = self.conv3(x)
        #print('conv3 called')   
        x = self.dropout(x)
        x = self.batchnorm2(x)
        #print('final main ',self.conv3.kernel_size,self.conv3.stride, x.shape)

        # Summing main and side branches
        x = x + x_copy
        x = self.prelu3(x)
        #print('final total ', x.shape)
        
        return x

In [ ]:
class ENet_encoder(nn.Module):
  
  # Creating Enet model!
  
    def __init__(self, C):
        super().__init__()
        
        # Define class variables
        # C - number of classes
        self.C = C
        
        # The initial block
        self.init = InitialBlock()
        
        # The first bottleneck
        self.b10 = RDDNeck(dilation=1, 
                           in_channels=16, 
                           out_channels=64, 
                           down_flag=True, 
                           p=0.01)
        
        self.b11 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b12 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b13 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        self.b14 = RDDNeck(dilation=1,
                           in_channels=64, 
                           out_channels=64, 
                           down_flag=False, 
                           p=0.01)
        
        
        # The second bottleneck
        self.b20 = RDDNeck(dilation=1, 
                           in_channels=64, 
                           out_channels=128, 
                           down_flag=True)
        
        self.b21 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b22 = RDDNeck(dilation=2, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b23 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b24 = RDDNeck(dilation=4, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b25 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b26 = RDDNeck(dilation=8, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b27 = ASNeck(in_channels=128, 
                          out_channels=128)
        self.b28 = RDDNeck(dilation=16, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        
        # The third bottleneck
        self.b31 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b32 = RDDNeck(dilation=2, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b33 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b34 = RDDNeck(dilation=4, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b35 = RDDNeck(dilation=1, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b36 = RDDNeck(dilation=8, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        self.b37 = ASNeck(in_channels=128, 
                          out_channels=128)
        
        self.b38 = RDDNeck(dilation=16, 
                           in_channels=128, 
                           out_channels=128, 
                           down_flag=False)
        
        #fully convolutional layer to get the encoder output
        self.enc_conv = nn.Conv2d(in_channels=128,
                                 out_channels = 12,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0,
                                 bias =False)
    def forward(self, x):
        
        # The initial block
        x = self.init(x)
        #print('^^^^^^^^^^^^^^init block^^^^^^^^^^^^^^^^')
        
        # The first bottleneck
        x, i1 = self.b10(x)
        #print('................bottleneck_10 complete................')
        x = self.b11(x)
        #print('................bottleneck_11 complete...........')
        x = self.b12(x)
        #print('...............bottleneck_12 complete............')
        x = self.b13(x)
        #print('............bottleneck_13 complete............')
        x = self.b14(x)
        #print('.................bottleneck_14 complete.............')
        
        # The second bottleneck
        x, i2 = self.b20(x)
        #print('----------------bottleneck_20 complete-----------')
        x = self.b21(x)
        #print('----------------bottleneck_21 complete-----------')
        x = self.b22(x)
        #print('-----------bottleneck_22 complete--------------')
        x = self.b23(x)
        #print('-----------bottleneck_23 complete---------------')
        x = self.b24(x)
        #print('--------------bottleneck_24 complete------------')
        x = self.b25(x)
        #print('-------------bottleneck_25 complete------------')
        x = self.b26(x)
        #print('-------------bottleneck_26 complete-------------')
        x = self.b27(x)
        #print('-------------bottleneck_27 complete-------------')
        x = self.b28(x)
        #print('------------bottleneck_28 complete----------------')
        
        # The third bottleneck
        x = self.b31(x)
        #print('********bottleneck_31 complete************')
        x = self.b32(x)
        #print('********bottleneck_32 complete************')
        x = self.b33(x)
        #print('********bottleneck_33 complete************')
        x = self.b34(x)
        #print('********bottleneck_34 complete************')
        x = self.b35(x)
        #print('********bottleneck_35 complete************')
        x = self.b36(x)
        #print('********bottleneck_36 complete************')
        x = self.b37(x)
        #print('********bottleneck_37 complete************')
        x = self.b38(x)
        #print('********bottleneck_38 complete************')
        

        x = self.enc_conv(x)
      
        
        return x, i1,i2
        

## Instantiate the ENet model

Move the model to cuda if available

In [ ]:
# Checking if there is any gpu available and pass the model to gpu or cpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

encoder = ENet_encoder(12)
encoder.to(device)

In [ ]:
print(device)

## Define the class weights (only for cross-entropy loss)

In [ ]:
class_weights = [ 6.09752836 , 4.50045501 ,31.41213452 , 3.44543644 ,14.96523731 , 8.93200049,
 29.91471566 ,30.77730888 ,12.69374868 ,37.01958742 ,41.13626664 ,18.56930107]

## visualising pipe

In [ ]:
def verify_dataset_normalization(loader,h=360,w=480):
    """finds mean and std dev for whole data batch
    """
    # Initialize accumulators
    channel_sum = torch.tensor([0.0, 0.0, 0.0])
    channel_squared_sum = torch.tensor([0.0, 0.0, 0.0])
    num_batches = 0

    for images, _ in train_loader:
        #print(images.shape)
        # Accumulate sum and squared sum for each channel
        channel_sum += images.sum(dim=[0, 2, 3])
        channel_squared_sum += (images ** 2).sum(dim=[0, 2, 3])
        num_batches += images.shape[0]

    # Calculate mean and standard deviation
    mean = channel_sum / (num_batches * h * w)
    std = (channel_squared_sum / (num_batches * h * w) - mean ** 2) ** 0.5

    return mean, std

In [ ]:
def augmentation(mode='train', h= 360, w =480):
    """ resizes image to input size and mask to a downsampled size, 
        applies horizontal flip and color jitter augmentation only to trainsets
        
    """
    if mode == 'train':
        img_transformation = A.Compose([
                        A.Resize(h,w),
                        A.HorizontalFlip(p= 0.5),
                        #A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),

                    ])
        #mask size different from image size
        mask_transformation = A.Compose([
                    A.Resize(h//8, w//8),
                    A.HorizontalFlip(p=0.5)
                    ])
        
    else:
        img_transformation =A.Resize(h, w)
        mask_transformation = A.Resize(h//8, w//8)
        
    return img_transformation,  mask_transformation

In [ ]:
#img = cv2.imread('/kaggle/input/CamVid/train/', cv2.COLOR_BGR2RGB)
#mask = cv2.imread()

In [ ]:
class CamvidDataset(Dataset):
    """custom camvid datset that returns images and the corresponding masks after augmentation and normalisation  
        
    """
    def __init__(self, img_path, mask_path, augmentation, norm_transform=True, road_idx=None):
        self.filenames_t = os.listdir(img_path)
        self.img_path = img_path
        self.mask_path = mask_path
        self.norm_transform = norm_transform
        self.augmentation = augmentation
        #for exttracting road mask
        #self.road_idx = road_idx 

    def __len__(self):
        return len(self.filenames_t)

    def __getitem__(self, idx):
        each_img_path = os.path.join(self.img_path, self.filenames_t[idx])
        each_mask_path = os.path.join(self.mask_path, self.filenames_t[idx])
        img = cv2.imread(each_img_path, cv2.COLOR_BGR2RGB)
        label_array = cv2.imread(each_mask_path, cv2.IMREAD_GRAYSCALE)#h,w array datatype
        label = np.expand_dims(label_array, axis = -1)#h,w,c
        
        if self.augmentation:
            img_transforms, mask_transforms = self.augmentation
            seed = 7
            random.seed(seed)
            img = img_transforms(image=img)['image']# albumentations must be passed with named argument, and gets stored with that name as key
            random.seed(seed)
            mask = mask_transforms(image=label)['image']
            
        if self.norm_transform:
            normalize_tensor = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.4119, 0.4251, 0.4327], 
                                                [0.3047, 0.3096, 0.3054])
                                     ])
            img = normalize_tensor(img)
            
        if not isinstance(mask, torch.Tensor):
            mask = torch.tensor(mask, dtype=torch.uint8)
        mask = mask.permute(2, 0, 1)#c,h,w
        #print(mask.shape)
        mask = mask.squeeze()#h,w
        #print(mask.shape)

        return img, mask



In [ ]:
class Idd7ClassesDataset(Dataset):
    def __init__(self, img_path, mask_path, augmentation, norm_transform=True, road_idx=None):
        self.filenames_t = glob.glob(training_path+f'*\*.jpg')
        self.filenames_s = glob.glob(segmented_path+f'*\*[0-9]_label.png')
        self.norm_transform = norm_transform
        self.dual_transform = dual_transform
        self.road_idx = road_idx

    def __len__(self):
        return len(self.filenames_t)

    def __getitem__(self, idx):
        img_path = self.filenames_t[idx]
        mask_path = self.filenames_s[idx]
        img = cv2.imread(each_img_path, cv2.COLOR_BGR2RGB)
        label_array = cv2.imread(each_mask_path, cv2.IMREAD_GRAYSCALE)#h,w array datatype
        label = np.expand_dims(label_array, axis = -1)#h,w,c
        
        if self.augmentation:
            img_transforms, mask_transforms = self.augmentation
            seed = 7
            random.seed(seed)
            img = img_transforms(image=img)['image']# albumentations must be passed with named argument, and gets stored with that name as key
            random.seed(seed)
            mask = mask_transforms(image=label)['image']
            
        if self.norm_transform:
            normalize_tensor = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.3576, 0.3713, 0.3657], 
                                                [0.2608, 0.2723, 0.2943])
                                     ])
            img = normalize_tensor(img)
            
        if not isinstance(mask, torch.Tensor):
            mask = torch.tensor(mask, dtype=torch.uint8)
        mask = mask.permute(2, 0, 1)#c,h,w
        #print(mask.shape)
        mask = mask.squeeze()#h,w
        #print(mask.shape)

        return img, mask


In [ ]:
# train_dataset = CamvidDataset('/kaggle/input/CamVid/train/','/kaggle/input/CamVid/trainannot/',augmentation('train'))
# val_dataset = CamvidDataset('/kaggle/input/CamVid/val/','/kaggle/input/CamVid/valannot/',augmentation('val'))
# test_dataset = CamvidDataset('/kaggle/input/CamVid/test/','/kaggle/input/CamVid/testannot/',augmentation('test'))

train_dataset = Idd7ClassesDataset("/kaggle/input/idd-lite1-6k/idd20k_lite/leftImg8bit/train/",
                                    "/kaggle/input/idd-lite1-6k/idd20k_lite/gtFine/train/", augmentation('train'))
val_dataset = Idd7ClassesDataset("/kaggle/input/idd-lite1-6k/idd20k_lite/leftImg8bit/val/",
                                    "/kaggle/input/idd-lite1-6k/idd20k_lite/gtFine/val/",augmentation('val'))

# train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False) 
    
# mean_train, std_train = verify_dataset_normalization(train_loader)
# mean_val, std_val = verify_dataset_normalization(val_dataset)
# mean_test, std_test = verify_dataset_normalization(test_loader)

# print(f'mean and std dev of train set is {mean_train}, {std_train}')
# print(f'mean and std dev of val set is {mean_val}, {std_val}')
# print(f'mean and std dev of test set is {mean_test}, {std_test}')


In [ ]:
def decode_segmap(image, threshold=0.5):
    
    #print(image)#RGB
    Sky = [0, 0, 0]
    Building = [0, 0, 153]
    Pole = [0, 0, 255]
    Road = [51, 153, 255]
    Pavement = [0, 255, 255]
    Tree = [128, 255, 0]
    SignSymbol = [255, 255, 0]
    Fence = [64, 64, 128]
    Car = [255, 128, 0]
    Pedestrian = [255, 0, 127]
    Bicyclist = [255, 204, 255]
    Background_scene = [255,255,255]

    label_colours = np.array([Sky, Building, Pole, Road, 
                              Pavement, Tree, SignSymbol, Fence, Car, 
                              Pedestrian, Bicyclist, Background_scene]).astype(np.uint8)
    
    print(label_colours.shape)
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
    
    for l in range(0, 12):
        r[image == l] = label_colours[l, 0]
        g[image == l] = label_colours[l, 1]
        b[image == l] = label_colours[l, 2]

    rgb = np.zeros((image.shape[0], image.shape[1], 3)).astype(np.uint8)
    rgb[:, :, 0] = r
    rgb[:, :, 1] = g
    rgb[:, :, 2] = b
    return rgb


In [ ]:
import matplotlib.pyplot as plt

def visualize_sample(dataset, idx):
    img, mask = dataset[idx]  # Fetch the image and mask using the dataset's __getitem__ method
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(img.permute(1, 2, 0))  # Assuming img is a PyTorch tensor of shape [C, H, W]
    ax[0].set_title("Image")
    ax[1].imshow(mask, cmap='gray')  # Assuming mask is a PyTorch tensor of shape [C, H, W] and C=1 for grayscale
    ax[1].set_title("Mask")
    plt.show()

# Example usage
visualize_sample(train_dataset, 5)  # Visualize the first sample in the train dataset


In [ ]:
idx = 0

img, mask = train_dataset[idx]
print(img.shape, mask.shape)
#print(mask)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
        
ax1.set_title('IMAGE')
ax1.imshow(img.permute(1,2,0).squeeze())
        
ax2.set_title('GROUND TRUTH')
ax2.imshow(mask.squeeze(),cmap = 'gray')
mask_segmap =decode_segmap(mask.squeeze().numpy())

ax2.imshow(mask_segmap)

## Define the Hyperparameters

In [ ]:
from torchmetrics.classification import MulticlassJaccardIndex
jaccard = MulticlassJaccardIndex(num_classes=12).to(device)


## Training loop

In [ ]:

# bc_train = 367 // batch_size # count of mini_batch train
# bc_eval = 101 // batch_size  # count of mini_batch validation
# bc_test = 233 // batch_size

# epochs = 20
# save_every = 200
total_run :int = 1

In [ ]:
def log_image_table(X_batch, outputs, mask_batch):
    table = wandb.Table(columns = ["images", "predictions", "targets"] 
            )
    
    for X_img, output, mask in zip(X_batch.to("cpu"), outputs.to("cpu"), mask_batch.to("cpu")):
        segmap_pred = decode_segmap(output.data.max(0)[1].numpy())#pass the max prob channel
        segmap_gt = decode_segmap(mask.numpy())
        
        table.add_data(wandb.Image(X_img.numpy().transpose(1,2,0)*255), 
                  wandb.Image(segmap_pred), 
                  wandb.Image(segmap_gt)
                 )
    wandb.log({"predictions_table": table}, commit = False) #lets commit to the server with the metrics

In [ ]:
def validation_loop(model, val_loader, criterion, log_images = False, batch_idx = 0):
    """ params: model, val_loader,loss function, 
        flag log_images: to decide to log the predictions in table
        flag batch_idx : to log a single batch consistently over all epochs, default 0th batch, 
        
        returns validation loss and accuracy averaged for that epoch """
    model.eval()
    val_loss = 0
    
    with torch.inference_mode():
        cum_jacc = 0
        for step, (X_batch, mask_batch)in enumerate(val_loader):
            
            X_batch, mask_batch = X_batch.to(device), mask_batch.to(device)
            outputs,_,_ = model(X_batch.float())
            val_loss += criterion(outputs, mask_batch.long())* X_batch.size(0)
            cum_jacc += jaccard(outputs, mask_batch)
            
            if step == batch_idx and log_images:
                log_image_table(X_batch, outputs, mask_batch)
        return val_loss/len(val_loader.dataset), cum_jacc/len(val_loader.dataset)
            
        

In [ ]:
for run in range(total_run):
    config ={"learning_rate": 5e-4,
                        "batch_size": 10,
                        "epochs": 100,
                       } 
    wandb.init(project ='enet_idd_lite_encoder_training',
                name=f'lr={config["learning_rate"]}_bs={config["batch_size"]}_epochs={config["epochs"]}',
               config=config)

    config = wandb.config
    
    train_loader = DataLoader(train_dataset, config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, config.batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, config.batch_size, shuffle=False) 
    
    n_steps_per_epoch = math.ceil(len(train_dataset)/config.batch_size)
    model = encoder
    criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights).to(device))
    optimizer = torch.optim.Adam(encoder.parameters(), 
                             lr=config.learning_rate,
                             weight_decay=2e-4)
    
    
    model.to(device)
    element_step_ct = 0
    step_ct = 0
    for e in range(config.epochs):
          
        model.train()
    
        for step, (X_batch,mask_batch) in enumerate(train_loader):
            
            # assign data to cpu/gpu
            X_batch, mask_batch = X_batch.to(device), mask_batch.to(device) 
            out,i1,i2 = model(X_batch.float())

            optimizer.zero_grad()
            train_loss = criterion(out, mask_batch.long())
            
            # update weights
            train_loss.backward()
            optimizer.step()
            element_step_ct += len(X_batch)
            
            train_metrics = {"train/loss": train_loss,#loss per batch
                       "train/epoch": (step +1 +(n_steps_per_epoch*e))/n_steps_per_epoch,
                       "train:element_step_ct":element_step_ct
                      }
            if step+1<n_steps_per_epoch:
                wandb.log(train_metrics)
            
            step_ct +=1
        
        val_loss, jacc_acc = validation_loop(model, val_loader, criterion, log_images = (e == (config.epochs-1))) 
        val_metrics = {"val/val_loss": val_loss,
                      "val/val_accuracy": jacc_acc}
        
        ##logging val metrics alongside train metrics
        wandb.log({**train_metrics, **val_metrics })
        
        #print final achieved losses and accuracy at the end of each epoch
        print(f"Train loss: {train_loss:.3f},Valid_loss: {val_loss:.3f},Accuracy: {jacc_acc:.2f}")
        
        if e == (config.epochs-1):
            checkpoint = {
            'epochs' : e,
            'state_dict' : model.state_dict(),
            'maxpool_indices_1':i1,
            'maxpool_indices_2':i2,
            'jacc':jacc_acc,
            }
            torch.save(checkpoint, './encoder idd {}th epoch, indices with state_dic.pth'.format(e))
            print ('Model is saved!')
            
wandb.finish()

In [ ]:
############################# VALID LOADER ##############################
img_aug_func, mask_aug_func = Augmentation('val')

dual_transform = DualTransform(img_aug_func, mask_aug_func)

transform_norm = None
#transforms.Compose([
    #transforms.ToTensor(),
    #transforms.Normalize(mean=[0.4119, 0.4251, 0.4327], std=[0.3047, 0.3097, 0.3054])
#])

# Instantiate your dataset
test_dataset = CustomDataset('/kaggle/input/CamVid/test/',
                               '/kaggle/input/CamVid/testannot/',3 , 
                               norm_transform=transform_norm, dual_transform=dual_transform)

# Create your data loader
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

# Now you can iterate over train_loader to calculate mean and standard deviation
mean_test, std_test = verify_dataset_normalization(test_loader)
print('mean and std dev of test set :', mean_test,std_test)



In [ ]:

def test_model(model, epochs,test_loaders, criterion, optimizer,device):
    jacc_t_acc =[]   
    test_losses =[]
    for e in range(1,epochs+1):
        model.to(device)
        with torch.no_grad():
            model.eval()
            
            test_loss = 0
            cum_test_iou =0
            cum_jacc_t = 0
            # Validation loop
            for inputs, labels in tqdm(val_loader):
                
                inputs, labels = inputs.to(device), labels.squeeze(1).to(device)
                    
                out, id1,id2 = model(inputs)
                loss = criterion(out, labels.long())
                test_loss += loss.item()
                preds_ohe = torch.argmax(out, dim=1)
                  
                #cum_test_iou += calculate_mIoU(preds_ohe, labels, 2) 
                target = labels.squeeze(1)         
                cum_jacc_t += jaccard(preds_ohe,target)

        test_losses.append(test_loss/bc_test)
        #test_acc.append(cum_test_iou/batch_size)
        jacc_t_acc.append(cum_jacc_t/bc_test)
        print ('Epoch {}/{}...'.format(e, epochs),
                
                'test_Loss {:6f}'.format(test_losses[-1]),
                #'test_acc {:6f}'.format(test_acc[-1]),
                'jacc_t_acc {:6f}'.format(jacc_t_acc[-1]))
        
    print ('Epoch {}/{}...'.format(e, epochs),
           'Total Mean test loss:{:6f}'.format(sum(test_losses)/epochs))
    return  test_losses, jacc_t_acc
      
      

In [ ]:
test_losses, test_acc = test_model(encoder, epochs,test_loader, criterion, optimizer, device)

In [ ]:
print(test_acc)

In [ ]:
print(test_losses)

In [ ]:

# Move the test_losses tensor to the CPU before converting to NumPy array
plt.plot(train_losses, label = 'train_losses')
plt.plot(eval_losses, label='valid_losses')
plt.plot(test_losses, label='test_losses')
plt.title('Valid loss and test loss for Cross-Entropy')
plt.xlabel('epochs')
plt.ylabel(' CE')
plt.legend()
plt.show()


In [ ]:
print(eval_acc)
print(test_acc)

In [ ]:
# Function to ensure all elements are tensors and on the CPU
def prepare_for_plotting(data_list):
    processed_list = []
    for item in data_list:
        # Check if the item is a tensor
        if isinstance(item, torch.Tensor):
            # Move tensor to CPU and convert to numpy, then to scalar
            processed_item = item.cpu().item()
        else:
            # Item is already a scalar
            processed_item = item
        processed_list.append(processed_item)
    return processed_list

# Prepare eval_acc and test_acc for plotting
eval_acc_plot = prepare_for_plotting(eval_acc)
test_acc_plot = prepare_for_plotting(test_acc)

# Now you can plot
plt.plot(eval_acc_plot, label='val_accuracy')
plt.plot(test_acc_plot, label='test_accuracy')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend(frameon=True)
plt.title('Validation and Test Accuracy trend combined')
plt.show()


In [ ]:
plt.plot( test_losses,label='test loss')
plt.plot( test_acc_plot,label='test acc')
plt.legend(frameon=True)
plt.title('test acc and test loss')
plt.show()

In [ ]:
plt.plot( eval_losses,label='val loss')
plt.plot( eval_acc_plot,label='val acc')
plt.legend(frameon=True)
plt.title('val acc and val loss')
plt.show()

## Infer using the trained model

In [ ]:
h = 360
w = 480

image_path = '/kaggle/input/CamVid/test/0001TP_008730.png'
# Assuming the dataset is camvid

tmg_ = plt.imread(image_path)
tmg_ = cv2.resize(tmg_, (h, w), cv2.INTER_NEAREST)
tmg = torch.tensor(tmg_).unsqueeze(0).float()
tmg = tmg.transpose(2, 3).transpose(1, 2)
tmg = tmg.to(device)

with torch.no_grad():
    out1, id1, id2 = encoder(tmg.float())
    out1 = out1.squeeze(0)
b_ = out1.data.max(0)[1].cpu().numpy()

decoded_segmap = decode_segmap(b_)

gt_path = image_path.replace('test','testannot') 
gt = plt.imread(gt_path)
gt = cv2.resize(gt, (h, w), cv2.INTER_NEAREST)

images = {
 0 : ['Input Image', tmg_],
 1 : ['Predicted Segmentation', decoded_segmap],
 2 : ['ground truth', gt]
}

plt.title('cross_entropy_multi-class_80_epochs')

show_images(images)


In [ ]:
checkpoint = {
            'epochs' : epochs,
            'state_dict' : encoder.state_dict(),
            'train_error' :train_losses,
            'val_error' : eval_losses,
            'iou_trend' : eval_acc,
            'optimizer': optimizer.state_dict()
            #'scheduler': scheduler.state_dict()
            
            }
torch.save(checkpoint, '/kaggle/working/ckpt-enet-camvid_CE-{}-.pth'.format(100))
print ('Model saved!')
            

In [ ]:
checkpoint = torch.load('/kaggle/working/ckpt-enet-camvid-ce-12-100-21.237043470144272.pth')
enet = ENet(12)
enet = enet.to(device)
state_dict = checkpoint['.squeeze()']
enet.load_state_dict(state_dict)

## Manual calculation of mIoU

In [ ]:
def intersection_over_union(pred_mask, true_mask):
    intersection = torch.logical_and(pred_mask, true_mask).sum().item()
    union = torch.logical_or(pred_mask, true_mask).sum().item()

    iou = intersection / union if union > 0 else 0.0
    return iou

def calculate_mIoU(predictions, targets, num_classes):
    class_iou = [0] * num_classes
    
    for class_idx in range(num_classes):
        pred_mask = (predictions == class_idx)
        true_mask = (targets == class_idx)
        class_iou[class_idx] = intersection_over_union(pred_mask, true_mask)

    mean_iou_value = sum(class_iou) / num_classes
    
    #print('road iou',class_iou[4])
    return mean_iou_value

